In [2]:
db_id = ''
db_pw = ''
db_nm = ''

In [4]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import requests
import time
from tqdm import tqdm
import os

import pymysql
from sqlalchemy import create_engine
from sqlalchemy import text
import re
import datetime
import json

import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup

# DB관련 함수정의
def db_connection(host_name=host_name):
    host_url = host_url
    user_nm = db_id
    passwd = db_pw
    port_num = port_num
    db_name = db_nm
    conn = pymysql.connect(host=host_url, user=user_nm, passwd=passwd, port = port_num, charset='utf8', cursorclass=pymysql.cursors.DictCursor) # cursorclass=pymysql.cursors.DictCursor 추가     # db=db,
    return conn

def engine_upload(upload_df: object, host_name: object = 'ds', db_name: object = db_name, table_name: object = table_name) -> object:
    host_url = host_url
    user_nm = db_id
    passwd = db_pw
    port_num = port_num
    db_name = db_nm
    engine = create_engine(f'mysql+pymysql://{user_nm}:{passwd}@{host_url}:{port_num}/{db_name}?charset=utf8mb4')
    engine_conn = engine.connect()
    upload_df.to_sql(table_name, engine_conn, if_exists='append', index=None)
    engine_conn.close()
    engine.dispose()
    
directory = os.getcwd()
print("Current working directory : ", os.getcwd())

#수집날짜
folder_date = str(datetime.datetime.now().date())
folder_date = re.sub("-", ".", folder_date)
today = re.sub("\.", "", folder_date)
print('오늘 날짜 : ', folder_date)

In [251]:
conn = db_connection(host_name)
book_href = pd.read_sql(f"select * from {table_name} where status is null;", conn)
conn.close()

In [252]:
book_href.head()

,id,genre,category_list,category_rank,title,utm_code,book_href,status,regist_date
0,140,art,Best Book Cover Art,140,Miss Peregrine's Home for Peculiar Children (M...,9460487,https://www.goodreads.com/book/show/9460487-mi...,None,NaT
1,141,art,Best Book Cover Art,141,Wintergirls,5152478,https://www.goodreads.com/book/show/5152478-wi...,None,NaT
2,142,art,Best Book Cover Art,142,Flow Down Like Silver,6801755,https://www.goodreads.com/book/show/6801755-fl...,None,NaT
3,143,art,Best Book Cover Art,143,The Replacement,7507908,https://www.goodreads.com/book/show/7507908-th...,None,NaT
4,144,art,Best Book Cover Art,144,"Crescendo (Hush, Hush, #2)",7791997,https://www.goodreads.com/book/show/7791997-cr...,None,NaT


In [253]:
print(len(book_href))

84872


In [179]:
drv = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [1]:
for i in tqdm(range(0, len(book_href))):
    book_df = pd.DataFrame(columns = ['genre', 'genre2', 'category_list', 'category_rank', 'utm_code', 'title', 
                                  'original_title', 'isbn13', 'isbn10', 'writer', 'publisher', 'publish_date', 
                                  'page', 'rating', 'price', 'language', 'description', 'character', 'awards', 
                                  'cover', 'setting', 'image'])
    drv.get(book_href['book_href'][i])
    
    # login popup
    try:
        drv.find_elements(By.CLASS_NAME, 'Icon.CloseIcon')[-1].click()
    except:
        pass
    drv.implicitly_wait(10)
    soup = BeautifulSoup(drv.page_source, 'html.parser')
    
    # 책 표지, 가격, 작가명, 평점, 출판일
    try:
        price = float(soup.find('div', {'class':'BookActions'}).find_all('div', {'class':'BookActions__button'})[-1].text.split('$')[-1])
    except:
        drv.get(book_href['book_href'][i])
        drv.implicitly_wait(10)
        soup = BeautifulSoup(drv.page_source, 'html.parser')
        
    image = soup.find('img', {'class':'ResponsiveImage'})['src']
    writer = soup.find('span', {'class':'ContributorLink__name'}).text
    rating = float(soup.find('div', {'class':'RatingStatistics__rating'}).text)
    
    detail_box = soup.find('div', {'class':'BookDetails'})
    publish_date = detail_box.find('div', {'class':'FeaturedDetails'}).find('p', {'data-testid':'publicationInfo'}).text.split('published')[-1].strip()
    
    # 장르 펼치기
    try:
        drv.find_element(By.CLASS_NAME, 'BookPageMetadataSection__genres').find_elements(By.CLASS_NAME, 'Button.Button--tag-inline.Button--small')[-1].click()
    except:
        drv.get(book_href['book_href'][i])
        time.sleep(1)
        drv.find_element(By.CLASS_NAME, 'BookPageMetadataSection__genres').find_elements(By.CLASS_NAME, 'Button.Button--tag-inline.Button--small')[-1].click()

    # book detail 펼치기
    try:
        drv.find_element(By.CLASS_NAME, 'BookDetails').find_element(By.CLASS_NAME, 'Button__labelItem').click()
    except:
        drv.get(book_href['book_href'][i])
        time.sleep(1)
        drv.find_element(By.CLASS_NAME, 'BookDetails').find_element(By.CLASS_NAME, 'Button__labelItem').click()        
    
    # 페이지, 커버, 출판사, isbn13, isbn10, language
    detail_box = drv.find_element(By.CLASS_NAME, 'BookDetails')
    edit_list = detail_box.find_element(By.CLASS_NAME, 'EditionDetails').find_elements(By.CLASS_NAME, 'DescListItem')
    
    page, cover, publisher, isbn13, isbn10, language  = 0, '', '', '', '', ''
    edit_text_list = []
    for e in range(len(edit_list)):
        edit_text_list.append(edit_list[e].text.split('\n'))

    for et in range(len(edit_text_list)):
        if edit_text_list[et][0] == 'Format':
            format_list = edit_text_list[et][1].split(' pages, ')
        elif edit_text_list[et][0] == 'Published':
            publisher = edit_text_list[et][1].split('by')[-1].strip()
        elif edit_text_list[et][0] == 'ISBN':
            isbn_list = edit_text_list[et][1].split('(')
            try: # 없거나 하나만 있는 경우도 있음..
                isbn13, isbn10 = isbn_list[0].strip(), isbn_list[1].replace('ISBN10: ', '').replace(')', '').strip()
            except:
                if len(isbn_list[0].strip())==13:
                    isbn13 = isbn_list[0].strip()
                else:
                    isbn10 = isbn_list[1].replace('ISBN10: ', '').replace(')', '').strip()
        elif edit_text_list[et][0] == 'Language':
            language = edit_text_list[et][1]
    
    try: # 없는 경우도 있음..
        page, cover = int(format_list[0]), format_list[1]
    except:
        try:
            page = int(format_list[0])
        except:
            cover = format_list[0]
        
    # 줄거리
    description = ''
    description_soup = soup.find('div', {'class':'DetailsLayoutRightParagraph__widthConstrained'}).find('span', {'class':'Formatted'})
    for string in description_soup.stripped_strings:
        description += (repr(string)[1:-1] + ' ')
        
    # 장르
    genre = drv.find_element(By.CLASS_NAME, 'BookPageMetadataSection__genres').text.split('\n')[:-1]
    genre.remove('Genres')
    
    # 수상내역, 오리지널 타이틀, 세팅 국가, 캐릭터명 수집을 위한 접근
    try:
        WorkDetails = detail_box.find_element(By.CLASS_NAME, 'WorkDetails')
    except:
        WorkDetails = detail_box.find_element(By.CLASS_NAME, 'EditionDetails')
    
    # award show more click
    try:
        WorkDetails.find_element(By.CLASS_NAME, 'TruncatedContent__gradientOverlay').find_element(By.CLASS_NAME, 'Button__container').click()
    except:
        pass

    # 수상내역, 오리지널 타이틀, 세팅 국가, 캐릭터명
    desc_list = WorkDetails.find_elements(By.CLASS_NAME, 'DescListItem')

    desc_text_list = []
    for d in range(len(desc_list)):
        desc_text_list.append(desc_list[d].text.split('\n'))

    awards, original_title, setting, character = '', '', '', ''

    for dt in range(len(desc_text_list)):
        if desc_text_list[dt][0] == 'Literary awards':
            awards = desc_text_list[dt][1].split(', ')
        elif desc_text_list[dt][0] == 'Original title':
            original_title = desc_text_list[dt][1]
        elif desc_text_list[dt][0] == 'Setting':
            setting = desc_text_list[dt][1].split(', ')
        elif desc_text_list[dt][0] == 'Characters':
            character = desc_text_list[dt][1].split(', ')
    
    data = pd.DataFrame({'genre':book_href['genre'][i], 'genre2':[genre], 'category_list':book_href['category_list'][i], 
                         'category_rank':book_href['category_rank'][i], 'utm_code':book_href['utm_code'][i], 
                         'title':book_href['title'][i],'original_title':original_title, 'isbn13':isbn13, 
                         'isbn10':isbn10, 'writer':writer, 'publisher':publisher, 'publish_date':publish_date, 
                         'page':page, 'rating':rating, 'price':price, 'language':language, 
                         'description':description, 'character':[character], 'awards':[awards], 
                         'cover':cover, 'setting':[setting], 'image':image})
    book_df = pd.concat([book_df, data], ignore_index=True)
    
    book_df['genre2'] = book_df['genre2'].apply(lambda x : str(x))
    book_df['character'] = book_df['character'].apply(lambda x : str(x))
    book_df['awards'] = book_df['awards'].apply(lambda x : str(x))
    book_df['setting'] = book_df['setting'].apply(lambda x : str(x))
    
    # insert
    host_url = host_url
    port_num = port_num
    engine = create_engine(f'mysql+pymysql://{db_id}:{db_pw}@{host_url}:{port_num}/{db_nm}?charset=utf8mb4')
    engine_conn = engine.connect()

    book_df.to_sql(f'{upload_table_name}', con=engine, if_exists='append', index=None)
    engine_conn.close()
    engine.dispose()
    
    # 수집한 category의 status 변경
    conn = db_connection(host_name)
    cur = conn.cursor()
    
    cur.execute(
        f'update {table_name} set status = "fin" where id = "{book_href["id"][i]}"'
    )
    conn.commit()
    
    cur.close()
    conn.close()
    time.sleep(1)

In [203]:
book_df

,genre,genre2,category_list,category_rank,utm_code,title,original_title,isbn13,isbn10,writer,...,page,rating,price,language,description,character,awards,cover,setting,image
0,art,"['Fantasy', 'Young Adult', 'Romance', 'Paranor...",Best Book Cover Art,11,256683,"City of Bones (The Mortal Instruments, #1)",The Mortal Instruments,9781416914280,1416914285,Cassandra Clare,...,485,4.07,9.99,English,When fifteen-year-old Clary Fray heads out to ...,"['Alexander ""Alec"" Lightwood', 'Jonathan ""Jace...",['Locus Award Nominee for Best First Novel (20...,Hardcover,"['New York City', 'New York (United States', '...",https://images-na.ssl-images-amazon.com/images...
1,art,"['Fantasy', 'Young Adult', 'Romance', 'Paranor...",Best Book Cover Art,12,7171637,"Clockwork Angel (The Infernal Devices, #1)",Clockwork Angel,9781416975861,1416975861,Cassandra Clare,...,479,4.31,10.99,English,In a time when Shadowhunters are barely winnin...,"['Theresa ""Tessa"" Gray', 'Magnus Bane', 'Willi...",['RITA Award by Romance Writers of America Nom...,Hardcover,"['London', 'England (', '1878)']",https://images-na.ssl-images-amazon.com/images...
2,art,"['Horror', 'Fiction', 'Fantasy', 'Zombies', 'H...",Best Book Cover Art,13,5899779,Pride and Prejudice and Zombies (Pride and Pre...,Pride and Prejudice and Zombies,9781594743344,1594743347,Seth Grahame-Smith,...,320,3.31,9.99,English,“It is a truth universally acknowledged that a...,"['Mr. Bennet', 'Mrs. Bennet', 'Jane Bennet', '...",['The Kitschies Nominee for Red Tentacle (Nove...,Paperback,['United Kingdom'],https://images-na.ssl-images-amazon.com/images...
3,art,"['Young Adult', 'Fantasy', 'Science Fiction', ...",Best Book Cover Art,14,1656001,"The Host (The Host, #1)",The Host,9780316068048,0316068047,Stephenie Meyer,...,624,3.86,8.99,English,Melanie Stryder refuses to fade away. The eart...,"['Melanie Stryder', ""Ian O'Shea"", 'Wanderer', ...",,Hardcover,['Arizona (United States)'],https://images-na.ssl-images-amazon.com/images...
4,art,"['Fantasy', 'Young Adult', 'Romance', 'Paranor...",Best Book Cover Art,15,305234,"Wicked Lovely (Wicked Lovely, #1)",Wicked Lovely,9780061214653,0061214655,Melissa Marr,...,328,3.70,9.99,English,Rule #3: Don't stare at invisible faeries. Ais...,"['Aislinn', 'Keenan', 'Leslie', 'Irial', 'Evan...",['RITA Award by Romance Writers of America for...,Hardcover,"['Huntsdale', 'Pennsylvania (United States)']",https://images-na.ssl-images-amazon.com/images...
5,art,"['Young Adult', 'Angels', 'Fantasy', 'Romance'...",Best Book Cover Art,16,7778981,"Halo (Halo, #1)",Halo,9780312656263,0312656262,Alexandra Adornetto,...,485,3.66,7.99,English,An angel is sent to Earth on a mission. But fa...,"['Bethany Church', 'Xavier Woods', 'Gabriel Ch...",,Hardcover,,https://images-na.ssl-images-amazon.com/images...
6,art,"['Fantasy', 'Young Adult', 'Historical Fiction...",Best Book Cover Art,17,3682,"A Great and Terrible Beauty (Gemma Doyle, #1)",A Great and Terrible Beauty,9780689875342,0689875347,Libba Bray,...,403,3.79,9.99,English,"In this debut gothic novel mysterious visions,...","['Gemma Doyle', 'Kartik', 'Ann Bradshaw', 'Fel...","['Iowa High School Book Award (2007)', 'Lincol...",Hardcover,"['United Kingdom', 'India (', '1895)', 'London...",https://images-na.ssl-images-amazon.com/images...
7,art,"['Fiction', 'Historical Fiction', 'Romance', '...",Best Book Cover Art,18,43641,Water for Elephants,Water for Elephants,9781565125605,1565125606,Sara Gruen,...,368,4.11,11.99,English,"When Jacob Jankowski, recently orphaned and su...","['Jacob Jankowski', 'August (Water for Elephan...",['Book Sense Book of the Year Award for Adult ...,Paperback,"['The United States of America', 'Joliet', 'Il...",https://images-na.ssl-images-amazon.com/images...
8,art,"['Classics', 'Fiction', 'Historical Fiction', ...",Best Book Cover Art,19,2657,To Kill a Mockingbird,To Kill a Mockingbird,9781565125605,1565125606,Harper Lee,...,323,4.26,11.99,English,The unforgettable novel of a childhood in a sl...,"['Scout Finch